In [2]:
# save models
import numpy as np # TF-IDF 
import pickle
import jieba 
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from tensorflow.keras.preprocessing.sequence import pad_sequences

C:\Users\lenovo\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lenovo\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lenovo\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# train the new model

In [3]:
# Load the dataset
balanced_df_all_cleaned_tokenization = pd.read_csv('balanced_df_all_cleaned.csv')

In [5]:
balanced_df_all_cleaned_tokenization

,ID,label,tweet_content
0,25640,0,害真挺吃颜想岁足表达减肥决心姐妹谈恋爱感觉全世界闺蜜介绍成功行淦近新情侣装评手评寻思俩关系挺...
1,28303,0,拥抱世界暖心动作生活中难免挫折坎坷安慌乱许拥抱充满量世界抗衡愿生山树栖心爱春赏花夏纳凉秋登山...
2,10546,0,雨天穿滑倒坐水里亲测艾特子赟家意见麻雀妈妈问麻雀扎头发麻雀说啾啾晚子赟帮辫头发感找年糊图张回...
3,26929,0,幸福做桌菜然家朋友吃完没帮忙刷碗老规矩昨晚想做早餐奶包发酵忘遛弯回馒头样子朋友吃午两点吃早新...
4,31080,0,日清明青草疫情中牺牲医护员公安干警基层干部线工作逝世胞沉痛悼念情病毒面前选择逆行选择死生选择...
...,...,...,...
20645,10321,1,意做件错事真久久原谅近发生事情绪波动房间爆哭然雨天跑出外面冻两时爸妈姨表哥电话接妈妈外面找找...
20646,10322,1,太喜欢张教授说话特实危言耸听然整体医生亲戚朋友医生医生子女微博关注医生说话客观尤专业领域戴口...
20647,10323,1,毒药断送爱听说爱情蜜糖深陷沉醉岂知份爱情美意想原理结束意想称爱情毒药爱情终止测试图点外卖点外...
20648,10324,1,果原谅痛苦值降低贼悔吃麻辣串洗完澡新换衣服全味难受新闻男熬夜猝死死女抑郁症确诊天昨天加药量睡...


In [4]:
# Replace NaN values with an empty string
balanced_df_all_cleaned_tokenization['tweet_content'] = balanced_df_all_cleaned_tokenization['tweet_content'].fillna('')


In [6]:
df2=balanced_df_all_cleaned_tokenization.copy()

In [10]:
df_label_0 = df2[df2['label'] == 0].sample(n=500, random_state=43)
df_label_1 = df2[df2['label'] == 1].sample(n=500, random_state=43)

# 合并两个dataframe
df3 = pd.concat([df_label_0, df_label_1])


In [25]:
# df3=df2.copy()

In [11]:
df3

,ID,label,tweet_content
8426,24791,0,车王徐浪浪哥电影想红玩微拍暴力明天午两点半直播总发车仪式明天午两点半直播总发车仪式路走老脆弱...
10,18973,0,分享单曲喜欢原唱陈洁仪课代表估计追青刷朋友圈文案觉生畏头秃做分钟报告拍张低头发际线堪忧片放进...
10042,31405,0,魔族炼气士刚刚祭灵未扑头老龟便见道道剑气落半空中象头鹰头独眼魔神头蛛魔神头头颅雨般落选李彤拒...
6144,15303,0,卡日签领红包签越红包越限时奖励快领错惜微博红包祝生日快乐湖北黄氏考察调研记浦仲诚隐梅斋湖北黄...
3002,13335,0,轉發微博轉發微博工作王道刘昊然张慧雯林姑娘腰功厉害王源确认出演玄幻电视剧主宰男主角牧尘首位担...
...,...,...,...
14730,4406,1,果样负面情绪丢真实情况想吞百颗药安心睡长长醒觉长特想回抱抱告诉洪水猛兽情绪然理解早晨点吃药睡...
19578,9254,1,忙早点床晚点半前班记前段时间站视频谈学校里抑郁杀学长宿舍楼跳血肉模糊丝意识哭说想死求求救救终...
15078,4754,1,知心理健康情况做心理测试表现出智障健忘杀意愿计划残太残年月份告张丽化名寻甸县疾病预防控制中心...
11063,739,1,胃酸流星难受嗝屁谜叔十点送入昏迷然三点醒痛苦流星夜做事聊便开播差点谜叔送昏迷流星夜切实感岛噪...


# 新训练数据集

In [ ]:
# Preprocess the data
X = df3['tweet_content']
y = df3['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Tokenize the sequences using Jieba tokenizer
def tokenize(text):
    return list(jieba.cut(text))

X_train_sequences = [tokenize(x) for x in X_train]
X_test_sequences = [tokenize(x) for x in X_test]

# Convert tokens to integer sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train_sequences)

X_train_sequences = tokenizer.texts_to_sequences(X_train_sequences)
X_test_sequences = tokenizer.texts_to_sequences(X_test_sequences)

# Pad sequences
max_sequence_length = 7000  # Set the maximum sequence length for padding
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

# Define the model architecture
embedding_dim = 100
num_filters = 100
filter_sizes = [3, 4, 5]

inputs = tf.keras.Input(shape=(max_sequence_length,))
embedding_layer = tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(inputs)
conv_layers = []
for filter_size in filter_sizes:
    conv_layer = tf.keras.layers.Conv1D(num_filters, filter_size, activation='relu')(embedding_layer)
    pool_layer = tf.keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers.append(pool_layer)
concatenated = tf.keras.layers.Concatenate()(conv_layers)
dropout = tf.keras.layers.Dropout(0.5)(concatenated)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_padded, y_test))

# Evaluate the model
y_pred = model.predict(X_test_padded)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)

Epoch 1/10
22/22 [==============================] - 22s 970ms/step - loss: 0.6882 - accuracy: 0.5414 - val_loss: 0.6626 - val_accuracy: 0.7600
Epoch 2/10
22/22 [==============================] - 21s 961ms/step - loss: 0.5755 - accuracy: 0.8929 - val_loss: 0.5901 - val_accuracy: 0.8600
Epoch 3/10
22/22 [==============================] - 21s 965ms/step - loss: 0.4331 - accuracy: 0.9714 - val_loss: 0.4282 - val_accuracy: 0.9467
Epoch 4/10
22/22 [==============================] - 22s 994ms/step - loss: 0.2510 - accuracy: 0.9757 - val_loss: 0.2567 - val_accuracy: 0.9500
Epoch 5/10
22/22 [==============================] - 22s 1s/step - loss: 0.1297 - accuracy: 0.9800 - val_loss: 0.1886 - val_accuracy: 0.9433
Epoch 6/10
22/22 [==============================] - 22s 1s/step - loss: 0.0855 - accuracy: 0.9857 - val_loss: 0.1675 - val_accuracy: 0.9433
Epoch 7/10
22/22 [==============================] - 21s 971ms/step - loss: 0.0651 - accuracy: 0.9914 - val_loss: 0.1599 - val_accuracy: 0.9500
Epoch

In [29]:
# Save the model
tf.saved_model.save(model, 'TextCNN_0807test200')

INFO:tensorflow:Assets written to: TextCNN_0807test200\assets


INFO:tensorflow:Assets written to: TextCNN_0807test200\assets


# f20230807 解决save model问题

In [8]:
# Preprocess the data
X = df3['tweet_content']
y = df3['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Tokenize the sequences using Jieba tokenizer
def tokenize(text):
    return list(jieba.cut(text))

X_train_sequences = [tokenize(x) for x in X_train]
X_test_sequences = [tokenize(x) for x in X_test]

# Convert tokens to integer sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train_sequences)

X_train_sequences = tokenizer.texts_to_sequences(X_train_sequences)
X_test_sequences = tokenizer.texts_to_sequences(X_test_sequences)

# Pad sequences
max_sequence_length = 7000  # Set the maximum sequence length for padding
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.637 seconds.
Prefix dict has been built successfully.


In [42]:
# Define the model architecture
embedding_dim = 100
num_filters = 100
filter_sizes = [3, 4, 5]

inputs = tf.keras.Input(shape=(max_sequence_length,))
embedding_layer = tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(inputs)
conv_layers = []
for filter_size in filter_sizes:
    conv_layer = tf.keras.layers.Conv1D(num_filters, filter_size, activation='relu')(embedding_layer)
    pool_layer = tf.keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers.append(pool_layer)
concatenated = tf.keras.layers.Concatenate()(conv_layers)
dropout = tf.keras.layers.Dropout(0.5)(concatenated)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_padded, y_test))

# Evaluate the model
y_pred = model.predict(X_test_padded)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)



Epoch 1/10
38/38 [==============================] - 59s 2s/step - loss: 0.6636 - accuracy: 0.6450 - val_loss: 0.5937 - val_accuracy: 0.9139
Epoch 2/10
38/38 [==============================] - 44s 1s/step - loss: 0.4295 - accuracy: 0.9175 - val_loss: 0.2755 - val_accuracy: 0.9211
Epoch 3/10
38/38 [==============================] - 44s 1s/step - loss: 0.1680 - accuracy: 0.9408 - val_loss: 0.1688 - val_accuracy: 0.9311
Epoch 4/10
38/38 [==============================] - 43s 1s/step - loss: 0.1102 - accuracy: 0.9525 - val_loss: 0.1502 - val_accuracy: 0.9368
Epoch 5/10
38/38 [==============================] - 54s 1s/step - loss: 0.0761 - accuracy: 0.9742 - val_loss: 0.1466 - val_accuracy: 0.9400
Epoch 6/10
38/38 [==============================] - 62s 2s/step - loss: 0.0502 - accuracy: 0.9883 - val_loss: 0.1376 - val_accuracy: 0.9418
Epoch 7/10
38/38 [==============================] - 58s 2s/step - loss: 0.0374 - accuracy: 0.9917 - val_loss: 0.1329 - val_accuracy: 0.9432
Epoch 8/10
38/38 [==

In [38]:
# Save the model with explicit tracing
@tf.function(input_signature=[tf.TensorSpec(shape=(None, max_sequence_length), dtype=tf.int32)])
def predict_fn(x):
    return model(x)

tf.saved_model.save(model, 'TextCNN_0807test2003', signatures={'serving_default': predict_fn})

INFO:tensorflow:Assets written to: TextCNN_0807test2003\assets


INFO:tensorflow:Assets written to: TextCNN_0807test2003\assets


In [43]:

# Save the model using model.save()
model.save('TextCNN_0807test2007.h5')


In [45]:

# Load the model
loaded_model = tf.keras.models.load_model('TextCNN_0807test2007.h5')

# Evaluate the loaded model
y_pred_loaded = loaded_model.predict(X_test_padded)
y_pred_binary_loaded = (y_pred_loaded > 0.5).astype(int)

accuracy_loaded = accuracy_score(y_test, y_pred_binary_loaded)
f1_loaded = f1_score(y_test, y_pred_binary_loaded)
recall_loaded = recall_score(y_test, y_pred_binary_loaded)
precision_loaded = precision_score(y_test, y_pred_binary_loaded)

print("Loaded Model Accuracy:", accuracy_loaded)
print("Loaded Model F1 Score:", f1_loaded)
print("Loaded Model Recall:", recall_loaded)
print("Loaded Model Precision:", precision_loaded)


88/88 [==============================] - 19s 212ms/step
Loaded Model Accuracy: 0.9446428571428571
Loaded Model F1 Score: 0.9435748088824172
Loaded Model Recall: 0.9303661162957645
Loaded Model Precision: 0.9571639586410635


# Loading the model

In [2]:
# Load the dataset
balanced_df_all_cleaned_tokenization = pd.read_csv('balanced_df_all_cleaned_tokenization.csv')

In [3]:
df3=balanced_df_all_cleaned_tokenization.copy()

In [4]:
# Replace NaN values with an empty string
balanced_df_all_cleaned_tokenization['tweet_content'] = balanced_df_all_cleaned_tokenization['tweet_content'].fillna('')
df2=balanced_df_all_cleaned_tokenization.copy()
df_label_0 = df2[df2['label'] == 0].sample(n=2000, random_state=42)
df_label_1 = df2[df2['label'] == 1].sample(n=2000, random_state=42)

# 合并两个dataframe
df3 = pd.concat([df_label_0, df_label_1])


In [5]:
df3 

,ID,label,tweet_content
10148,31839,0,手中 行 原油 宝 失误 客户 买单 国油板 价想 生日 月 日 祝福 事 花开花落 难堪 ...
1692,22295,0,北极 绒纯 棉裤 条卷 评拍 送条 共 发条 肠道 提高 孩子 免疫力 卷 评晚 睡觉 抱 ...
2692,17168,0,早安 周季 松茸 价格低 家斤 囤 优惠 特价 家 囤 祝 生日快乐 喝点 水朵 姣 云南 ...
7703,21714,0,式样 友纸 召唤 训导 快 平安 京 富豪 灯烬 熄烛 阴阳 全新 阶式 神 浮世 青行 灯...
321,29150,0,做 学习型 妈妈 建设 学习 成长型 家庭 建立 家庭 精神 宝藏 免费 围观 回答 价值 ...
...,...,...,...
11487,1163,1,真 难受 关注 转发 抽 粉丝 送 国风 衣裙 条条 爱爱 裙子 胸口 红花 刺绣 吸睛 裙...
14887,4563,1,清明 长假 三天 家 两天 出浪 憋 久 天气 错 穿 汉服 浪 两天 家 睡睡 睡睡 吃饭...
11996,1672,1,生理 趋势 杀 害怕 双相 情感 障碍 事 折磨 吃饭 哭泣 闭 抑郁 傻瓜 做事 然偶 傻...
14184,3860,1,花邑 永远 神 加劲 克制 抑郁 失位 亲 爷爷 年位 姑年 忌日 四天 前 三月初 三 三...


In [6]:
# Preprocess the data
X = df3['tweet_content']
y = df3['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)#0.2 for test, 0.3 for whole

# Tokenize the sequences using Jieba tokenizer
def tokenize(text):
    return list(jieba.cut(text))

In [7]:
# Convert any float values in X_train and X_test to strings
X_train = [str(x) for x in X_train]
X_test = [str(x) for x in X_test]



In [8]:
X_train_sequences = [tokenize(x) for x in X_train]
X_test_sequences = [tokenize(x) for x in X_test]

# Convert tokens to integer sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train_sequences)

X_train_sequences = tokenizer.texts_to_sequences(X_train_sequences)
X_test_sequences = tokenizer.texts_to_sequences(X_test_sequences)

# Pad sequences
max_sequence_length = 7000  # Set the maximum sequence length for padding
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.775 seconds.
Prefix dict has been built successfully.


In [9]:

# Load the model
loaded_model = tf.keras.models.load_model('TextCNN_0807test2007.h5')

# Evaluate the loaded model
y_pred_loaded = loaded_model.predict(X_test_padded)
y_pred_binary_loaded = (y_pred_loaded > 0.5).astype(int)

accuracy_loaded = accuracy_score(y_test, y_pred_binary_loaded)
f1_loaded = f1_score(y_test, y_pred_binary_loaded)
recall_loaded = recall_score(y_test, y_pred_binary_loaded)
precision_loaded = precision_score(y_test, y_pred_binary_loaded)

print("Loaded Model Accuracy:", accuracy_loaded)
print("Loaded Model F1 Score:", f1_loaded)
print("Loaded Model Recall:", recall_loaded)
print("Loaded Model Precision:", precision_loaded)

88/88 [==============================] - 20s 219ms/step
Loaded Model Accuracy: 0.9446428571428571
Loaded Model F1 Score: 0.9435748088824172
Loaded Model Recall: 0.9303661162957645
Loaded Model Precision: 0.9571639586410635


In [11]:
loaded_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 7000)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 7000, 100)    16086500    ['input_2[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 6998, 100)    30100       ['embedding_1[0][0]']            
                                                                                                  
 conv1d_4 (Conv1D)              (None, 6997, 100)    40100       ['embedding_1[0][0]']            
                                                                                            

In [9]:
# test model
# Load the model
loaded_model = tf.saved_model.load('TextCNN_0705whole')# TextCNN_whole2 TextCNN_test TextCNN_0705whole

# Convert the loaded model to a Keras model
model = loaded_model.signatures["serving_default"]

In [17]:
# test model 0807
# Load the model
loaded_model = tf.saved_model.load('TextCNN_0807test2002')# TextCNN_whole2 TextCNN_test TextCNN_0705whole

# Convert the loaded model to a Keras model
model = loaded_model.signatures["serving_default"]

In [13]:
# Set the batch size for evaluation
batch_size = 32

In [18]:
# Evaluate the model in batches
y_pred_binary = []
num_batches = int(np.ceil(len(X_test_padded) / batch_size))

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(X_test_padded))
    batch_data = X_test_padded[start_idx:end_idx]
    batch_data_tensor = tf.constant(batch_data, dtype=tf.float32)
    batch_output = model(batch_data_tensor)['dense']
    batch_pred = (batch_output > 0.5).numpy().astype(int)
    y_pred_binary.extend(batch_pred)

y_pred_binary = np.array(y_pred_binary)

InvalidArgumentError: cannot compute __inference_signature_wrapper_6441 as input #0(zero-based) was expected to be a int32 tensor but is a float tensor [Op:__inference_signature_wrapper_6441]

In [ ]:
# Calculate evaluation metrics
accuracy_TextCNN = accuracy_score(y_test, y_pred_binary)
f1_TextCNN = f1_score(y_test, y_pred_binary)
recall_TextCNN = recall_score(y_test, y_pred_binary)
precision_TextCNN = precision_score(y_test, y_pred_binary)

print("Accuracy:", accuracy_TextCNN)
print("F1 Score:", f1_TextCNN)
print("Recall:", recall_TextCNN)
print("Precision:", precision_TextCNN)

In [23]:
# Preprocess the new tweet

In [22]:
# normal content
# new_tweet_content = ['这是 一条 新的 正常 推文']

# depressed content
new_tweet_content = ['思诺思 舒乐安定 代开 抑郁 抑郁 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合盆友 解释 快 超话 传送门 南京 兼职']

new_tweet_sequences = [tokenize(tweet) for tweet in new_tweet_content]
new_tweet_sequences = tokenizer.texts_to_sequences(new_tweet_sequences)
new_tweet_padded = pad_sequences(new_tweet_sequences, maxlen=max_sequence_length, padding='post')

# Convert the padded sequences to float32
new_tweet_padded = new_tweet_padded.astype(np.float32)

# Make predictions
output = model(tf.constant(new_tweet_padded))  # Pass the data as a constant tensor
predictions = output['dense']  # Assuming the final layer in your model is named 'dense'
predicted_labels = (predictions > 0.5).numpy().astype(int)  # Convert the predictions to a numpy array

# Print the predicted labels
print("Predicted Labels:", predicted_labels)


Predicted Labels: [[0]]
